Install requred dependencies

In [1]:
!pip install transformers accelerate diffusers xformers safetensors opencv-python
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

Initialise stable diffusion xl pipeline

Create pipeline for text 2 image using Diffusers and Stable Diffusion xl

In [2]:
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler ,AutoPipelineForText2Image,ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers.utils import load_image
from PIL import Image
import torch
import numpy as np
import cv2
import torch

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/sdxl-turbo/blob/main/sd_xl_turbo_1.0_fp16.safetensors",
    torch_dtype=torch.float16, variant="fp16")
pipeline = pipeline.to("cuda")
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config, timestep_spacing="trailing")





The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sd_xl_turbo_1.0_fp16.safetensors:   0%|          | 0.00/6.94G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

Some weights of the model checkpoint were not used when initializing CLIPTextModelWithProjection: 
 ['text_model.embeddings.position_ids']


Initialising

In [4]:

controlnet_conditioning_scale = 0.5  # recommended for good generalization

controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0",
    torch_dtype=torch.float16
)
negative_prompt = 'low quality, bad quality, sketches'

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
controlNetPipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16,
)
controlNetPipe.enable_model_cpu_offload()


config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer_2/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer_2/tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Using Ngrok to forward port from google collab

In [3]:
from pyngrok import ngrok
authtoken = "2XCmmFBG2pe8Eyac9w6p8YNVIvt_5aUfkK7DqNe6AWmooWi2S"

In [5]:
ngrok.set_auth_token(authtoken)

In [8]:
import requests
from pyngrok import ngrok
from typing import List
import uvicorn
import io
import nest_asyncio
from fastapi import FastAPI, HTTPException,Body
from fastapi.responses import JSONResponse, StreamingResponse
from time import sleep
from fastapi.middleware.cors import CORSMiddleware
from io import BytesIO
from diffusers.utils import load_image
from PIL import Image

app = FastAPI()
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# Function to generate user logo
def text_to_image(text):
  image = pipeline(prompt=text, guidance_scale=0.0, num_inference_steps=1).images[0]
  return image

def logo_transformation(img_url,text):
  print(img_url,text)
  image = load_image(img_url)
  image = np.array(image)
  image = cv2.Canny(image, 100, 200)
  image = image[:, :, None]
  image = np.concatenate([image, image, image], axis=2)
  image = Image.fromarray(image)

  images = controlNetPipe(
    prompt=text, negative_prompt=negative_prompt, image=image, controlnet_conditioning_scale=controlnet_conditioning_scale,
    ).images
  return images

@app.post("/generate-banner")
async def generate_text(data: dict = Body(...)):
    try:
        text = data.get('input')
        if not text:
            raise HTTPException(status_code=400, detail="Input text is required")

        generated_image = text_to_image(text)

        # Convert PIL Image to bytes
        img_byte_array = BytesIO()
        generated_image.save(img_byte_array, format="PNG")
        img_byte_array.seek(0)

        # Return image as a streaming response
        return StreamingResponse(img_byte_array, media_type="image/png")
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        raise HTTPException(status_code=500, detail="Internal server error")
@app.post("/transform-logo")
async def generate_text(data: dict = Body(...)):
    try:
        text = data.get('input')
        image_url=data.get("image_url")
        if not text or not image_url:
            raise HTTPException(status_code=400, detail="Input text and image url is required")

        generated_logo = logo_transformation(text,image_url)

        # Convert PIL Image to bytes
        img_byte_array = BytesIO()
        generated_logo.save(img_byte_array, format="PNG")
        img_byte_array.seek(0)

        # Return image as a streaming response
        return StreamingResponse(img_byte_array, media_type="image/png")
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        raise HTTPException(status_code=500, detail="Internal server error")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
sleep(0.5)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


Public URL: https://182e-34-32-197-81.ngrok-free.app


INFO:     Started server process [959]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     14.139.241.214:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     14.139.241.214:0 - "GET /openapi.json HTTP/1.1" 200 OK
Error processing image: Incorrect path or URL. URLs must start with `http://` or `https://`, and beautiful hd ultrarealistic jungle 4K is not a valid path.
INFO:     14.139.241.214:0 - "POST /transform-logo HTTP/1.1" 500 Internal Server Error


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [959]


KeyboardInterrupt: 